# Compare-2: total patient counts are similar for consecutive timespans. If test fail, then check Compare-12 and Compare-13.

Description: check if total patient counts are similar for consecutive timespans. If test fail, then check Compare-12 and Compare-13.

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) npi:                    npi column
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:  total patient count.


def compare_2(db_name, table_name, bene_cnt):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # MySQL query to calculate total patient count
        query = ('''
                SELECT SUM({col1})
                FROM {db}.{t1};
        '''.format(db = db_name, t1 = table_name, col1 = bene_cnt))

        cur.execute(query)
        
        print ("The total {} of {}.{} is:".format(bene_cnt, db_name, table_name) + '\n')        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))

        cur.close()
        connection.close()            

## Test Example

In [3]:
compare_2('_amy', 'test_data_good', 'cnt_bene_id')

The total cnt_bene_id of _amy.test_data_good is:

5085


In [4]:
compare_2('_amy', 'test_data_bad1', 'cnt_bene_id')

The total cnt_bene_id of _amy.test_data_bad1 is:

9932


In [5]:
compare_2('_amy', 'test_data_bad2', 'cnt_bene_id')

The total cnt_bene_id of _amy.test_data_bad2 is:

4777


## Internal Data Example

Compute total patient of HOP Teaming 2014, 2015 and 2016.

#### The total patient count of HOP Teaming 2014 is:

In [4]:
compare_2('npi_hop_RQ14', 'directed', 'patient_count')

The total patient_count of npi_hop_RQ14.directed is:

4202171126


#### The total patient count of HOP Teaming 2015 is:

In [9]:
compare_2('npi_hop_RQ15', 'directed', 'patient_count')

The total patient_count of npi_hop_RQ15.directed is:

4194552053


#### The total patient count of HOP Teaming 2016 is:

In [10]:
compare_2('npi_hop_RQ16', 'directed', 'patient_count')

The total patient_count of npi_hop_RQ16.directed is:

5866538297


#### The total patient count of HOP Teaming 2017 is:

In [11]:
compare_2('npi_hop_RQ17', 'directed', 'patient_count')

The total patient_count of npi_hop_RQ17.directed is:

10001727587
